<a href="https://colab.research.google.com/github/ErikMayerPSU/AuroraNSL_2024/blob/main/Code/viewline/Plot_OP_13_Aurorasaurus_and_view_line_2025_0430_1400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install aacgmv2

In [22]:
import logging; logging.basicConfig(level=logging.INFO)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import aacgmv2 as aacgmv2

import pandas as pd

import numpy as np

plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 1

In [23]:
#data = [[43.1419945,	-77.62689505, 1, '2015-03-18 02:00:00+00'], [43.07432708,	-77.08215706, 1, '2015-03-18 02:00:00+00'], [44.75479816,	-85.64451196, 1, '2015-03-18 02:00:00+00'],
#        [45.47064381,	-93.73595456, 1, '2015-03-18 02:00:00+00'], [56.721425,	-111.3920839, 1, '2015-03-18 02:00:00+00'], [49.27290253,	-96.52831503, 1, '2015-03-18 02:00:00+00'],
#        [44.48052241,	-81.38310197, 1, '2015-03-18 02:00:00+00'], [45.08066817,	-87.65610678, 1, '2015-03-18 02:00:00+00']]

#df_aurorasaurus = pd.DataFrame(data, columns=['st_y', 'st_x', 'See_Aurora', 'time_start'])

#print(df_aurorasaurus)

In [24]:
df_aurorasaurus = pd.read_csv(r"https://raw.githubusercontent.com/ErikMayerPSU/AuroraNSL_2024/refs/heads/main/data/20150318_0200UT_web_observations_and_tweets.txt", header=None, nrows=23, sep='\s+', on_bad_lines='skip')

# Define new headers
eflux_headers1 = ["see_aurora","date" ,"time", "st_y", "st_x"]

# Set the column headers
df_aurorasaurus.columns = eflux_headers1

df_aurorasaurus

see_aurora        date         time       st_y        st_x
0            1  2015-03-18  02:00:00+00  66.064519  -18.660158
1            0  2015-03-18  02:00:00+00  39.053329  -75.511607
2            1  2015-03-18  02:00:00+00  54.548501   -3.591408
3            0  2015-03-18  02:00:00+00  39.235292  -75.722088
4            1  2015-03-18  02:00:00+00  43.141995  -77.626895
5            0  2015-03-18  02:00:00+00  39.933899  -74.939244
6            1  2015-03-18  02:00:00+00  40.035265  -78.032827
7            0  2015-03-18  02:00:00+00  38.953415  -75.824369
8            1  2015-03-18  02:00:00+00  54.503138   -8.201311
9            1  2015-03-18  02:00:00+00  43.074327  -77.082157
10           1  2015-03-18  02:00:00+00  43.643427  -85.276052
11           0  2015-03-18  02:00:00+00  39.950913  -75.159145
12           0  2015-03-18  02:00:00+00  42.428918  -71.623931
13           1  2015-03-18  02:00:00+00  42.985668  -84.563667
14           1  2015-03-18  02:00:00+00  43.079610  -73.791712
15           1  2015-03-18  02:00:00+00  44.754798  -85.644512
16           1  2015-03-18  02:00:00+00  45.470644  -93.735955
17           1  2015-03-18  02:00:00+00  56.721425 -111.392084
18           1  2015-03-18  02:00:00+00  49.272903  -96.528315
19           1  2015-03-18  02:00:00+00  44.480522  -81.383102
20           1  2015-03-18  02:00:00+00  45.080668  -87.656107
21           1  2015-03-18  02:00:00+00  43.888412  -78.594130
22           1  2015-03-18  02:00:00+00  55.362429   -7.424810

In [25]:
# Converting column data from CSV to lists
xcoords = df_aurorasaurus['st_x'].tolist()
ycoords = df_aurorasaurus['st_y'].tolist()
date_time = pd.to_datetime(df_aurorasaurus['time'])  #Store date and time in a variable

# Function to convert coordinates (Geo to AACGM)
def convert_to_aacgm(geo_lat, geo_lon, altitude, dtime):
    aacgm_lat, aacgm_lon, aacgm_r = aacgmv2.wrapper.convert_latlon(
        geo_lat, geo_lon, altitude, dtime, method_code='G2A'
    )
    return aacgm_lat, aacgm_lon, aacgm_r

# Lists to store results
aacgm_latitudes = []
aacgm_longitudes = []
aacgm_radii = []
mlt_values = []

# Convert each coordinate with corresponding time
for lat, lon, time in zip(ycoords, xcoords, date_time):
    out_lat, out_lon, out_r = convert_to_aacgm(lat, lon, 0, time)  #Assuming altitude = 0
    aacgm_latitudes.append(out_lat)
    aacgm_longitudes.append(out_lon)
    aacgm_radii.append(out_r)

    # Convert AACGM longitude to MLT
    mlt = aacgmv2.convert_mlt(out_lon, time, m2a=False)[0]
    mlt_values.append(mlt)

temp = pd.DataFrame({
    'ML': aacgm_latitudes,
    'MLT': mlt_values
}, index=df_aurorasaurus.index)  # Align index with original DataFrame

# Concatenate the original and new DataFrames horizontally
df_aurorasaurus = pd.concat([df_aurorasaurus, temp], axis=1)

df_aurorasaurus


<ipython-input-25-ac9bfcdb426f>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_time = pd.to_datetime(df_aurorasaurus['time'])  #Store date and time in a variable


see_aurora        date         time       st_y        st_x         ML  \
0            1  2015-03-18  02:00:00+00  66.064519  -18.660158  65.694513   
1            0  2015-03-18  02:00:00+00  39.053329  -75.511607  47.226088   
2            1  2015-03-18  02:00:00+00  54.548501   -3.591408  51.040039   
3            0  2015-03-18  02:00:00+00  39.235292  -75.722088  47.437707   
4            1  2015-03-18  02:00:00+00  43.141995  -77.626895  51.517723   
5            0  2015-03-18  02:00:00+00  39.933899  -74.939244  47.988810   
6            1  2015-03-18  02:00:00+00  40.035265  -78.032827  48.554911   
7            0  2015-03-18  02:00:00+00  38.953415  -75.824369  47.180097   
8            1  2015-03-18  02:00:00+00  54.503138   -8.201311  51.246127   
9            1  2015-03-18  02:00:00+00  43.074327  -77.082157  51.379401   
10           1  2015-03-18  02:00:00+00  43.643427  -85.276052  52.665795   
11           0  2015-03-18  02:00:00+00  39.950913  -75.159145  48.042418   
12           0  2015-03-18  02:00:00+00  42.428918  -71.623931  49.838728   
13           1  2015-03-18  02:00:00+00  42.985668  -84.563667  51.998865   
14           1  2015-03-18  02:00:00+00  43.079610  -73.791712  50.879167   
15           1  2015-03-18  02:00:00+00  44.754798  -85.644512  53.750257   
16           1  2015-03-18  02:00:00+00  45.470644  -93.735955  54.406400   
17           1  2015-03-18  02:00:00+00  56.721425 -111.392084  63.371446   
18           1  2015-03-18  02:00:00+00  49.272903  -96.528315  57.910144   
19           1  2015-03-18  02:00:00+00  44.480522  -81.383102  53.220163   
20           1  2015-03-18  02:00:00+00  45.080668  -87.656107  54.119668   
21           1  2015-03-18  02:00:00+00  43.888412  -78.594130  52.362919   
22           1  2015-03-18  02:00:00+00  55.362429   -7.424810  52.244595   

          MLT  
0    1.726111  
1   21.431811  
2    2.225533  
3   21.413687  
4   21.255367  
5   21.486803  
6   21.207655  
7   21.403309  
8    1.974946  
9   21.304960  
10  20.543807  
11  21.467184  
12  21.790695  
13  20.610023  
14  21.601985  
15  20.509909  
16  19.745201  
17  18.026301  
18  19.466998  
19  20.910899  
20  20.319492  
21  21.168714  
22   2.034038

In [26]:
#Round the values of ML to the nearest 0.5 value so they fit in the established bins
df_aurorasaurus['ML'] = (df_aurorasaurus['ML'] * 2).round() / 2

#Round the values of MLT to the nearest 0.5 value so they fit in the established bins
df_aurorasaurus['MLT'] = (df_aurorasaurus['MLT'] * 4).round() / 4

df_aurorasaurus

see_aurora        date         time       st_y        st_x    ML    MLT
0            1  2015-03-18  02:00:00+00  66.064519  -18.660158  65.5   1.75
1            0  2015-03-18  02:00:00+00  39.053329  -75.511607  47.0  21.50
2            1  2015-03-18  02:00:00+00  54.548501   -3.591408  51.0   2.25
3            0  2015-03-18  02:00:00+00  39.235292  -75.722088  47.5  21.50
4            1  2015-03-18  02:00:00+00  43.141995  -77.626895  51.5  21.25
5            0  2015-03-18  02:00:00+00  39.933899  -74.939244  48.0  21.50
6            1  2015-03-18  02:00:00+00  40.035265  -78.032827  48.5  21.25
7            0  2015-03-18  02:00:00+00  38.953415  -75.824369  47.0  21.50
8            1  2015-03-18  02:00:00+00  54.503138   -8.201311  51.0   2.00
9            1  2015-03-18  02:00:00+00  43.074327  -77.082157  51.5  21.25
10           1  2015-03-18  02:00:00+00  43.643427  -85.276052  52.5  20.50
11           0  2015-03-18  02:00:00+00  39.950913  -75.159145  48.0  21.50
12           0  2015-03-18  02:00:00+00  42.428918  -71.623931  50.0  21.75
13           1  2015-03-18  02:00:00+00  42.985668  -84.563667  52.0  20.50
14           1  2015-03-18  02:00:00+00  43.079610  -73.791712  51.0  21.50
15           1  2015-03-18  02:00:00+00  44.754798  -85.644512  54.0  20.50
16           1  2015-03-18  02:00:00+00  45.470644  -93.735955  54.5  19.75
17           1  2015-03-18  02:00:00+00  56.721425 -111.392084  63.5  18.00
18           1  2015-03-18  02:00:00+00  49.272903  -96.528315  58.0  19.50
19           1  2015-03-18  02:00:00+00  44.480522  -81.383102  53.0  21.00
20           1  2015-03-18  02:00:00+00  45.080668  -87.656107  54.0  20.25
21           1  2015-03-18  02:00:00+00  43.888412  -78.594130  52.5  21.25
22           1  2015-03-18  02:00:00+00  55.362429   -7.424810  52.0   2.00

In [27]:
# OP-13 data

df_eflux = pd.read_csv(r"https://raw.githubusercontent.com/ErikMayerPSU/AuroraNSL_2024/main/data/St%20patrick's%20day%202015/Eflux/20150318_0200UT_Eflux_e%2Call.txt", header=None, skiprows=1, nrows=7680, sep='\s+', on_bad_lines='skip')

# Define new headers
eflux_headers = ["MLT", "ML", "[mW m^-2]"]

# Set the column headers
df_eflux.columns = eflux_headers

#add column for visibility threshold (VT) T/F
df_eflux["VT"]= 0

#Set VT to 1 or 0 based on aurora visibility threshold of 1 mW m^-2
df_eflux.loc[df_eflux["[mW m^-2]"]>=1, "VT"] = 1

#add a column for the equatorial boundry (EB) and viewline (VL)
df_eflux["EB"] = 0 #initialize as 0 not "0"
df_eflux["VL"] = 0

last_j = 0 #initialize last j

#apply viewline equation to OP-13 data
for i in range(len(df_eflux["MLT"])):

  #find first instance of visibility threshold being 1 for each MLT value 0-24
  selected_rows = df_eflux.loc[(df_eflux["MLT"] == i/4) & (df_eflux["VT"] >= 1)]

  # Check that there is an instance of the VT being 1 in this value of MLT
  if not selected_rows.empty:
    j = selected_rows.index[0]

    #2/13/25 added if statement to attempt to remove outliars

    if (abs(df_eflux.loc[last_j,"ML"]-df_eflux.loc[j,"ML"]) < 1):



    # Set the EB column to show the first instance of visibility threshold being
    # 1 for each MLT value 0-24
      df_eflux.loc[j,"EB"] = 1
    # Set the viewline based on the EB VL equation
      df_eflux.loc[j-8,"VL"] = 1

    last_j = j

# Filter to get the MLT and ML coordinates of all rows where the VL is present
VL_df = df_eflux[df_eflux['VL'] == 1]
VL_df = VL_df[['MLT','ML']]

# Find missing values of MLT and add them to the VL dataframe with a ML of NaN
MLT_vals = df_eflux['MLT'].unique()
Missing_MLT = set(MLT_vals) - set(VL_df['MLT'])
VL_df = pd.concat([VL_df, pd.DataFrame({'MLT': list(Missing_MLT), 'ML': [np.nan] * len(Missing_MLT)})])

# Fix order and index
VL_df = VL_df.sort_values(by='MLT').reset_index(drop=True)


# Interpolate missing viewline points
# Use limit = 10 to avoid filling large gaps, such as the middle of the VL
VL_df = VL_df.interpolate(method='linear', axis=0, limit=10)
VL_df = VL_df.interpolate(method='linear', axis=0, limit_direction='both', limit_area='outside') # Ensure edges are filled

#Round the values of ML to the nearest 0.5 value so they fit in the established bins
VL_df['ML'] = (VL_df['ML'] * 2).round() / 2



#Create new row for interpolated VL in df_eflux
df_eflux['New_VL'] = 0

#Set New_VL to 1 for rows corresponding to the MLT and ML coordinates in VL_df
for i, row in VL_df.iterrows():
    mlt = row['MLT']
    ml = row['ML']
    if not pd.isna(ml):  # Skip rows where ML is NaN
        df_eflux.loc[(df_eflux['MLT'] == mlt) & (df_eflux['ML'] == ml), 'New_VL'] = 1

# Create an array of the New_VL column
array_eflux = df_eflux.to_numpy()
array_VIS = array_eflux[0:7680,6:7]

# Set visibilty to a "1" from the viewline to 90 MLT
for i in range(96):
    array_temp = array_VIS[(i*80):((i+1)*80),0:1]

    try:
        # Find the index of the first occurrence of 1 in array_temp
        idx = np.where(array_temp == 1)[0][0]  # Use np.where to find the index
        #print(idx)
        array_VIS[(i*80)+idx:((i+1)*80),0:1] = 1
        #for j in range(idx):
            #array_VL[i * 80 + (80 - j)] = 1 #Fixed index out of range error
            #print(i * 80 + (80 - j))
    except IndexError: #Changed to IndexError since np.where returns an empty array if the value is not found which leads to an IndexError
        logging.NullHandler

# Add the visibility array back to eflux_df
df_eflux['VIS'] = pd.DataFrame(array_VIS, columns=['VIS'])
df_eflux

MLT    ML  [mW m^-2]  VT  EB  VL  New_VL  VIS
0      0.00  50.0   0.009205   0   0   0       0  0.0
1      0.00  50.5   0.009303   0   0   0       0  0.0
2      0.00  51.0   0.005775   0   0   0       0  0.0
3      0.00  51.5   0.020590   0   0   0       0  0.0
4      0.00  52.0   0.000387   0   0   0       0  0.0
...     ...   ...        ...  ..  ..  ..     ...  ...
7675  23.75  87.5   0.023140   0   0   0       0  1.0
7676  23.75  88.0   0.022190   0   0   0       0  1.0
7677  23.75  88.5   0.023080   0   0   0       0  1.0
7678  23.75  89.0   0.016270   0   0   0       0  1.0
7679  23.75  89.5   0.027940   0   0   0       0  1.0

[7680 rows x 8 columns]

In [28]:
df_eflux['see_aurora'] = np.nan

for i, row in df_aurorasaurus.iterrows():
    mlt = row['MLT']
    ml = row['ML']
    df_eflux.loc[(df_eflux['MLT'] == mlt) & (df_eflux['ML'] == ml), 'see_aurora'] = df_aurorasaurus.loc[i, 'see_aurora']

df_eflux

MLT    ML  [mW m^-2]  VT  EB  VL  New_VL  VIS  see_aurora
0      0.00  50.0   0.009205   0   0   0       0  0.0         NaN
1      0.00  50.5   0.009303   0   0   0       0  0.0         NaN
2      0.00  51.0   0.005775   0   0   0       0  0.0         NaN
3      0.00  51.5   0.020590   0   0   0       0  0.0         NaN
4      0.00  52.0   0.000387   0   0   0       0  0.0         NaN
...     ...   ...        ...  ..  ..  ..     ...  ...         ...
7675  23.75  87.5   0.023140   0   0   0       0  1.0         NaN
7676  23.75  88.0   0.022190   0   0   0       0  1.0         NaN
7677  23.75  88.5   0.023080   0   0   0       0  1.0         NaN
7678  23.75  89.0   0.016270   0   0   0       0  1.0         NaN
7679  23.75  89.5   0.027940   0   0   0       0  1.0         NaN

[7680 rows x 9 columns]

In [32]:
# Reshape into a grid (MLT=rows, ML=columns)
grid = df_eflux.pivot(index='MLT', columns='ML', values='[mW m^-2]')

# Convert MLT to radians
theta = np.deg2rad(grid.index * 15)  # 15° per hour

#Invert ML since the plot was backward
ml = grid.columns.values
ml_inverted = ml.max() - ml

# Extend edges for pcolormesh
theta_edges = np.append(theta, theta[0] + 2*np.pi)  # Close the loop (24h)
ml_edges = np.append(ml_inverted, ml_inverted[-1] + np.diff(ml_inverted)[0])  # Extend edges


# Plot
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': 'polar'})
pc = ax.pcolormesh(
    theta_edges,
    ml_edges,
    grid.T,  # Transpose to match (ML, MLT)
    cmap='viridis',
    shading='auto'
)

ax.set_theta_zero_location('S')  # 0° at bottom
ax.set_theta_direction(1)       # Set correct direction for MLT

# Label MLT ticks
ax.set_xticks(np.linspace(0, 2*np.pi, 8, endpoint=False))
ax.set_xticklabels(['00', '03', '06', '09', '12', '15', '18', '21'])

# Label ML ticks
r_ticks = np.linspace(ml.min(), ml.max(), 5)
ax.set_yticks(ml.max() - r_ticks)
ax.set_yticklabels([f'{int(x)}°' for x in r_ticks])

# Add labels and colorbar
#ax.set_xlabel('Magnetic Local Time (MLT)')
#ax.set_ylabel('Magnetic Latitude (ML)')
plt.colorbar(pc, label='[mW m^-2]', pad=0.1)

# Add view line to plot

# Get the geomagnetic coordinates of view line
view_points = df_eflux[df_eflux['New_VL'] == 1]
view_MLT = view_points['MLT'].values
view_ML = view_points['ML'].values

# Convert to polar coordinates (same as main plot)
view_theta = np.deg2rad(view_MLT * 15)  # MLT to radians
view_ml_inverted = ml.max() - view_ML   # Invert ML

# Plot the view line in red
ax.plot(
    view_theta, view_ml_inverted,
    color='red', linewidth=2,
)

# Add the Aurorasaurus Data

# Get the geomagnetic coordinates of Aurorasaurus Data
visible = df_eflux[df_eflux['see_aurora'] == 1]
visible_MLT = visible['MLT'].values
visible_ML = visible['ML'].values

# Convert to polar coordinates (same as main plot)
visible_theta = np.deg2rad(visible_MLT * 15)  # MLT to radians
visible_ml_inverted = ml.max() - visible_ML   # Invert ML

# Plot the aurora confirmed in green
ax.scatter(
    visible_theta, visible_ml_inverted,
    color='lime', s=30,
)

# Get the geomagnetic coordinates of Aurorasaurus Data
not_visible = df_eflux[df_eflux['see_aurora'] == 0]
not_visible_MLT = not_visible['MLT'].values
not_visible_ML = not_visible['ML'].values

# Convert to polar coordinates (same as main plot)
not_visible_theta = np.deg2rad(not_visible_MLT * 15)  # MLT to radians
not_visible_ml_inverted = ml.max() - not_visible_ML   # Invert ML

# Plot the aurora confirmed not visible in red
ax.scatter(
    not_visible_theta, not_visible_ml_inverted,
    color='red', s=30,
)


plt.title('OP-13 Prediction With View Line')
plt.tight_layout()
plt.show()